In [ ]:
#library imports
import cv2
import numpy as np
import tensorflow as tf
import time
from datetime import datetime, timedelta
import csv
import os
import shutil
from scipy.ndimage import gaussian_filter
from scipy.spatial import distance
import math
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.lines import Line2D
import matplotlib.colors
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# user input parameters
filename = '7_10_swash_tv_07_07_2023_2_11940.0_slomo_1p9'
beach_slope = 1.9

# setup
tf.keras.backend.clear_session()
tf_model_version = "swash_tv_v14"
model = tf.keras.models.load_model(tf_model_version + "_model")
cap = cv2.VideoCapture(filename + ".mp4") # 
frame_rate = cap.get(cv2.CAP_PROP_FPS) #fps
slomo_rate = 6
flip_image_lr = True
m_per_pixel = 1/1000
img_padding = 25
frame_start = 0
frame_end = 1000000000000 # just a really big number so the whole video is processed
cap.set(1, frame_start)
IMG_SIZE_pre = 1520 # 760 original image size is 1520x358
IMG_SIZE_WID_PRE_CROP = 358
IMG_SIZE_WID_pre = 160
crop_flag = True
downsample_ratio = 0.5
if crop_flag == True:
    IMG_SIZE = int(IMG_SIZE_pre*downsample_ratio)
    IMG_SIZE_WID = int(IMG_SIZE_WID_pre*downsample_ratio)
else:
    IMG_SIZE = IMG_SIZE_pre
    IMG_SIZE_WID = IMG_SIZE_WID_pre

"""%%%%%%%%%%%%%%%%%%% MAIN WHILE LOOP %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""

frame_end_count = 0
cap.set(1, frame_start)
frame_reduced_list = []
print("Extracting frames...")
while(cap.isOpened()):

    ret, frame = cap.read()    
    if frame_end_count > frame_end:
        break
    
    if ret == True:
        
        if flip_image_lr == True:
            frame = cv2.flip(frame, 1)
        
        if crop_flag == True:
            # crop top section of image away
            frame = frame[IMG_SIZE_WID_PRE_CROP - IMG_SIZE_WID_pre:,:]
            wid = int(frame.shape[1] * downsample_ratio)
            hig = int(frame.shape[0] * downsample_ratio)
            dim = (wid, hig)
            frame_reduced = cv2.resize(frame, dim, interpolation=cv2.INTER_NEAREST_EXACT)
        else:
            frame_reduced = cv2.resize(frame, (IMG_SIZE_WID, IMG_SIZE))
        
        frame_reduced_list.append(frame_reduced)
        frame_end_count = frame_end_count + 1
    else:
        break

print("Processing frames through Unet model...")

predictions = []
frames_proc = 100
# do N frames at a time
for i in range(0, len(frame_reduced_list), frames_proc):
    if len(frame_reduced_list) - i < frames_proc:
        residual = len(frame_reduced_list) - i
        frame_red_list_tf = np.array(frame_reduced_list[i:i + residual])
        frame_red_list_tf = frame_red_list_tf / 255
        frame_red_list_tf = frame_red_list_tf.reshape(-1, IMG_SIZE_WID, IMG_SIZE, 3)
        predict_mtx = model.predict([frame_red_list_tf])
        # pad past the angle brackets
        predict_mtx[:,:,0:img_padding,:] = [0,1]
        predict_mtx[:,:,-img_padding:,:] = [0,1]
        predictions.extend(predict_mtx)
        
    else:
        frame_red_list_tf = np.array(frame_reduced_list[i:i + frames_proc])
        frame_red_list_tf = frame_red_list_tf / 255
        frame_red_list_tf = frame_red_list_tf.reshape(-1, IMG_SIZE_WID, IMG_SIZE, 3)
        predict_mtx = model.predict([frame_red_list_tf])
        # pad past the angle brackets
        predict_mtx[:,:,0:img_padding,:] = [0,1]
        predict_mtx[:,:,-img_padding:,:] = [0,1]
        predictions.extend(predict_mtx)

print("Unet processing finished, now post processing data...")

# https://stackoverflow.com/questions/43892506/opencv-python-rotate-image-without-cropping-sides
def rotate_image(mat, angle):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    """

    height, width = mat.shape[:2] # image shape has 3 dimensions
    # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape
    image_center = (width/2, height/2) 
    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)
    
    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0])
    abs_sin = abs(rotation_mat[0,1])
    
    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h), flags=cv2.INTER_NEAREST)
    return rotated_mat

# create a graph of the water surface depth at all times measured up from the bottom of the image.
x_vals_all = []
z_vals_all = []
t_vals_all = []
h_vals_all = []
mov_img_list = []
j = 0
for prediction in predictions:
    test_image = frame_reduced_list[j]
    
    # process the prediction into a colour map
    test_model_guess = np.array(np.argmax(prediction, axis=-1), dtype=np.uint8)
    test_model_guess = np.where(test_model_guess == 1, 255, test_model_guess)
    test_model_guess = np.where(test_model_guess == 0, 10, test_model_guess)
    test_model_guess = np.array(test_model_guess, dtype=np.uint8)
    
    # colour the mask image
    r_channel = np.array(test_model_guess)
    b_channel = np.array(test_model_guess)
    g_channel = np.array(test_model_guess)
    
    # foamWake
    r_channel[r_channel == 10] = 155
    b_channel[b_channel == 10] = 150
    g_channel[g_channel == 10] = 80
    
    colour_img = cv2.merge((b_channel,g_channel,r_channel))
    
    # produce a transparent overaly image of the mask on the original image for visualization purposes.
    colour_img = cv2.addWeighted(test_image, 0.9, colour_img, 0.2, 0)
    
    # rotate image
    colour_img = rotate_image(colour_img, beach_slope)
    
    mov_img_list.append(colour_img)
    
    test_model_guess = test_model_guess[:,img_padding:-img_padding]
    test_model_guess = rotate_image(test_model_guess, beach_slope)
    test_model_guess = np.where(test_model_guess == 0, 255, test_model_guess)
    test_model_guess_cols = test_model_guess.T
    
    x_list = []
    t_list = []
    z_list = []
    h_list = []
    # extract top pixel from every column of the mask
    # take x_rot_offset away if rotated by Beta
    x_rot_offset = int(test_image.shape[0]*np.sin(np.deg2rad(beach_slope)))
    x_pad_offset = int(img_padding*np.cos(np.deg2rad(beach_slope)))
    for col in range(x_rot_offset, len(test_model_guess_cols), 1):
        # get indexes of all 210 values
        water_surf = np.where(test_model_guess_cols[col] == 10)
        if len(water_surf[0]) > 0:
            # z values need to be from bottom of image up
            z_val = (test_model_guess.shape[0] - min(water_surf[0]))*m_per_pixel/downsample_ratio
            z_list.append(z_val)
            cv2.circle(colour_img, (col + x_pad_offset, min(water_surf[0])), radius=1, color=[0, 0, 255])
            h_list.append(z_val - (col - x_rot_offset)*np.tan(np.deg2rad(beach_slope))*m_per_pixel/downsample_ratio)
            
        else:
            z_list.append(0)
            h_list.append(0)
        
        x_list.append((col - x_rot_offset)*m_per_pixel/downsample_ratio)
        t_list.append(j*(1/frame_rate)/slomo_rate)
        
    x_vals_all.append(x_list)
    z_vals_all.append(z_list)
    t_vals_all.append(t_list)
    h_vals_all.append(h_list)
    j = j + 1

x_vals_all_flat = np.concatenate(x_vals_all).ravel()
t_vals_all_flat = np.concatenate(t_vals_all).ravel()
z_vals_all_flat = np.concatenate(z_vals_all).ravel()
h_vals_all_flat = np.concatenate(h_vals_all).ravel()

# save values in csv file
rows = zip(x_vals_all_flat, t_vals_all_flat, z_vals_all_flat, h_vals_all_flat)
with open(filename + "_data.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['x (m)', 't (s)', 'z (m)', 'h (m)'])
    for row in rows:
        writer.writerow(row)

fig=plt.figure()
ax=fig.add_subplot(111)
ax.set_ylabel('t (s)')
ax.set_xlabel('x (m)')
plt.gca().invert_yaxis()

sc = plt.scatter(x_vals_all_flat, t_vals_all_flat, c=h_vals_all_flat, vmin=0, vmax=0.15, cmap='nipy_spectral', s=0.05, marker='.')
cbar = plt.colorbar(sc)
cbar.set_label('depth (m)')
plt.show()

fig.savefig(filename + "_graph.png", bbox_inches='tight', dpi=300)

# create video
video_recorded = cv2.VideoWriter(filename + "_annotated.mp4",
                                 cv2.VideoWriter_fourcc(*'DIVX'), frame_rate,
                                 (mov_img_list[0].shape[1], mov_img_list[0].shape[0]))

print("Writing a video...")
for i in range(0, len(mov_img_list), 1):
    video_recorded.write(mov_img_list[i])
video_recorded.release()

print("Swash TV analysis finished, data has been saved as a depth coloured timestack.")
cap.release()
cv2.destroyAllWindows()

print("done")